dt 적용 X

In [1]:
import numpy as np
import random
from math import *
import cmath
from scipy.linalg import expm, fractional_matrix_power

#########################################################################

# pauli matrix 
sx = np.array([[0,  1],     [1, 0]])
sy = np.array([[0, -1j],   [1j, 0]])
sz = np.array([[1, 0],     [0, -1]])
s0 = np.array([[1, 0],      [0, 1]])

# parameters(detuning factor)
v0 = 0.02           # Arbitrary settings, Actual speed : 0.04rad/μs
d0 = 0.15           # Arbitrary settings, Actual speed : 0.30rad/μs
dt = 2.6 


# unitary operator
def unitary(dt, choice):
    
    # Select x,y-rotation direction.
    # [stay, +x, -x, +y, -y]
    choice_list = [0, 1, -1, 1, -1] 
    
    if choice < 3:
        # if choice = 0 ... only d0*sz
        Ham = (d0*sz+v0*choice_list[choice]*sx)
    else:
        Ham = (d0*sz+v0*choice_list[choice]*sy)

    # Creating a Unitary Operator for each of the four sections by Hamiltonian
    eigvals = np.linalg.eigh(Ham)[0]
    eigvecs = 1*np.linalg.eigh(Ham)[1]
    E = np.diag(eigvals)
    U_H = eigvecs.conj().T
    U_e = U_H.conj().T @ expm(-1j*E*dt) @ U_H
    
    return U_e


#########################################################################

# x-rotation operater
def Rx(theta):
    return np.matrix([  [cos(theta/2),    -1j*sin(theta/2)],
                        [-1j*sin(theta/2),    cos(theta/2)]])

# z-rotation operater
# Do not use Rz. Control by rotation only by Hamiltonian.
def Rz(phi): 
    return np.matrix([  [cos(phi/2)-1j*sin(phi/2),  0],
                        [0,  cos(phi/2)+1j*sin(phi/2)]])

# Calculating the Fidelity
def state_fidelity(rho_1, rho_2): 
    
    # rho_1(current state), rho_2(target state)
    # Calculate the fidelity after checking the dimensions of the two states.
    
    if np.shape(rho_1) != np.shape(rho_2):
        print("Dimensions of two states do not match.")
        return 0
    else:
        sqrt_rho_1 = fractional_matrix_power(rho_1, 1 / 2)
        fidelity = np.trace(fractional_matrix_power(sqrt_rho_1 @ rho_2 @ sqrt_rho_1, 1 / 2)) ** 2
        return np.real(fidelity)


#########################################################################

init_wave = np.array([[1], [0]])
irho_init = np.kron(init_wave,init_wave.conj().T)

U_0 = unitary(dt, 0)
U_1 = unitary(dt, 1)
U_2 = unitary(dt, 2)
U_3 = unitary(dt, 3)
U_4 = unitary(dt, 4)

pulse_list = [U_0, U_1, U_2, U_3, U_4]


def fidelity(rho, predicted_seq):

    # predicted_seq = np.argmax(predicted_seq, axis=-1)

    Uni = s0
    
    for i in predicted_seq:
        Uni = pulse_list[i] @ Uni

    irho_final = Uni @ irho_init @ Uni.conj().T
    
    F = (state_fidelity(irho_final, rho))
    
    return  F

In [2]:
# # global input_rho

# def find_best_seq(index, seq):
     
#      global input_rho
#      global temp_seq
#      global temp_fid

#      # return when index is the first element of seq
#      if (-index) >= len(seq):
#           print("   =========================================================")
#           print("   Sequence failed because the first element must be changed")
#           print("   Or No elements to convert in the sequence")
#           print("   =========================================================")
#           return seq
     
#      # index < 0
#      # front_seq = seq[:index]
#      # back_seq = seq[index+1:]
#      final_seq = seq
     
#      for i in range(5):            # 0 to 4 try all elements
#           final_seq[index] = i

#           fid = fidelity(input_rho, final_seq)

#           if fid > 99.99:          # return to condition satisfaction
#                return final_seq
#           elif fid > temp_fid:
#                temp_fid = fid
#                temp_seq = final_seq
               
#           if start == False and index < -1:
#                temp_seq = find_best_seq(index + 1, temp_seq, False)  # recursive function
     
#      if start == True:
#           final_seq = find_best_seq(index - 1, temp_seq, False)        # recursive function
#      else:
#           final_seq = temp_seq
     
#      return final_seq

In [3]:
from itertools import product

def find_best_seq(seq):
     
     global input_rho
     
     final_seq = []
     temp_seq = []
     temp_fid = 0
     
     # for i in range(-1, -(len(seq)+1), -1):
     for i in range(-1, -8, -1):
          
          if i < -len(seq):
               break
          
          front_seq = seq[:i]
          back_seq = seq[i:]
          
          for combination in product(range(5), repeat=len(back_seq)):
               temp_seq = front_seq + list(combination)
               
               fid = fidelity(input_rho, temp_seq)
               
               if fid > 99.99:
                    final_seq = temp_seq
                    return final_seq
               elif fid > temp_fid:
                    final_seq = temp_seq
                    temp_fid = fid

               # print("=============================")
               # print(seq)
               # print(temp_seq)
               # print(final_seq)
               # print("=============================")

     return final_seq

In [4]:
import time

def local_optimizer(input_rho, input_seq):

     first_fid = fidelity(input_rho, input_seq)

     if first_fid > 99.99:
          output_seq = input_seq
     else:
          start_time = time.time()
          
          output_seq = find_best_seq(input_seq)
          
          end_time = time.time()
          computing_t = end_time - start_time

     last_fid = fidelity(input_rho, output_seq)

     total_t = dt * len(output_seq)

     return output_seq, first_fid, last_fid, total_t, computing_t

In [5]:
import os
from datetime import datetime
import pandas as pd

# Create a directory to store the results
dir = 'LO_results_dir'
if not os.path.exists(dir):
    os.makedirs(dir)

# CSV file name setup
date = datetime.now()
printdate = date.strftime('%Y%m%d_%H%M%S')
# filename = "/ByLO_dt_" + str(dt) + "_" + printdate + '.csv'
filename = "/ByLO_dt_" + "None_" + printdate + '.csv'

# Create an empty DataFrame and write to CSV file
saved_df = pd.DataFrame(columns=["Case", 'gate length', 'Theta', 'Phi', 'dt', 'combination', 'fidelity',
                           'total time', 'computing time'])
saved_df.to_csv(dir + filename, index=False)

In [6]:
import pandas as pd

df = pd.read_csv('../Data/using/dt_2.6/ByAstar_dt_2.6_1016.csv')
df_shuffled = df.sample(frac=1).reset_index(drop=True)

# df_shuffled.head()

thetas = df_shuffled['Theta']
phis = df_shuffled['Phi']
lists = df_shuffled['combination']

In [7]:
case = 0

for input_theta, input_phi, input_sequence in zip(thetas, phis, lists):  

     target_U = Rz(input_phi) @ Rx(input_theta)
     input_rho = target_U @ irho_init @ target_U.conj().T
     
     output_sequence, first_fidelity, last_fidelity, total_time, computing_time = local_optimizer(input_rho, eval(input_sequence))

     case += 1
     output = [['case' + str(case), len(output_sequence), input_theta, input_phi, dt, output_sequence, 
                    last_fidelity, total_time, computing_time]]
    
     # Create DataFrame and append to CSV file
     saved_df = pd.DataFrame(output, columns=["Case", 'gate length', 'Theta', 'Phi', 'dt', 'combination', 
                                                  'fidelity', 'total time', 'computing time'])
     saved_df.to_csv(dir + filename, mode='a', header=False, index=False)

     # Result output
     print(f"""
     ------------------------------------------------------------------------------------------------------
     theta = {input_theta}
     phi = {input_phi}
     input_sequence : {input_sequence}
     first_fidelity : {first_fidelity}
     output_sequence : {output_sequence}
     last_fidelity : {last_fidelity}
     total_time : {total_time}
     computing_time : {computing_time}
     ------------------------------------------------------------------------------------------------------
     """)
     
     print(case, "/", len(df_shuffled))



     ------------------------------------------------------------------------------------------------------
     theta = 2.082621838587949
     phi = 1.798123158743712
     input_sequence : [1, 1, 3, 3, 2, 2, 4, 4, 1, 1, 3, 3, 2, 2, 4, 4, 1, 1, 3, 3, 2, 2, 2, 0, 4, 2, 1]
     first_fidelity : 0.9947905593863359
     output_sequence : [1, 1, 3, 3, 2, 2, 4, 4, 1, 1, 3, 3, 2, 2, 4, 4, 1, 1, 3, 3, 2, 2, 2, 2, 4, 4, 1]
     last_fidelity : 0.9995806535462969
     total_time : 70.2
     computing_time : 216.4790325164795
     ------------------------------------------------------------------------------------------------------
     
1 / 29528

     ------------------------------------------------------------------------------------------------------
     theta = 1.624563371
     phi = 1.100188055
     input_sequence : [1, 1, 3, 3, 2, 2, 4, 1, 3, 3, 3, 3, 2, 2, 4, 4, 1, 1]
     first_fidelity : 0.9918398789793896
     output_sequence : [1, 1, 3, 3, 2, 2, 4, 1, 3, 3, 3, 2, 2, 2, 4, 4, 1, 1]
 

KeyboardInterrupt: 